# Classificação de Data Streams com Concept Drift

Olá! Neste notebook explicarei como fazemos a classificação de Data Streams com Concept Drift em Python! Mas primeiro, vamos tentar enteder o que são Data Streams, e o que é Concept Drift.

## Data Streams

Quando vamos fazer alguma coisa relacionada a Machine Learning, utilizamos uma base de dados, certo? E essa base de dados geralmente vem em batches, ou seja, em lote. Já podemos separá-la entre treino e teste. Mas quando falamos de Data Streams, esses dados chegam a todo momento. E qual o ponto disso? Em algum momento no tempo, pode acontecer alguma mudança em como esses dados se comportam. É essa mudança de conceito que chamamos de Concept Drift.

## Concept Drift

Temos algumas maneiras de explicar o que é um Concept Drift. Em termos gerais, é quando algo que acontece normalmente no mundo real muda. Por exemplo, tivemos a pandemia há algum tempo, certo? Todos começamos a usar máscaras ao sair da rua. Agora imagine um algoritmo de reconhecimento facial. Será que ele ainda conseguirá reconhecer os rostos de máscara? Se você tem um celular com reconhecimento facial, sabe a resposta.

Não, ele não conseguirá reconhecer bem os rostos. Isso porque ele nunca viu aquele conceito antes. Então podemos simplesmente treinar o modelo sobre os rostos com máscara, certo? Sim, isso poder ser feito, mas agora vamos ver alguns problemas dessa abordagem.

## Recursos computacionais

Quando falamos de Data Streams, temos que ter em mente que os dados estão chegando a todo momento. Já consegue imaginar o problema? Suponha que sua empresa tenha os dados na nuvem, e você precisa pagar pelo armazenamento. Entendeu agora? Pois é, uma quantidade muito grande de dados pede um armazenamento maior, e no contexto de Data Streams isso pode acabar virando uma bola de neve.

Outra justificativa para limitar os recursos computacionais, é que estes foram feitos para ser bem aproveitados. É importante que se tenha uma administração eficiente destes recursos. Podemos citar tempo, memória, e tamanho da amostra.

E lembramos que, em algum momento, pode acontecer uma mudança de conceito, as coisas passarão a funcionar de forma diferente na vida real. Assim, se você mantiver dados que não ajudam o seu modelo a fazer as predições, mas atrapalham, faz sentido deixá-los de lado, certo? E esse é um dos pontos que precisamos levar em conta quando trabalhamos com Concept Drift, o de que temos um limite de recursos para trabalhar.

# Concept Drift em Python

Temos uma biblioteca para trabalhar nesse cenário em Python, chamada Scikit Multiflow. Ela tem uma API semelhante à do Scikit Learn. A documentação está disponível em https://scikit-multiflow.github.io/. Existem algumas universidades que trabalham para mantê-lo, dentre elas a Universidade de Waikato, de onde saiu o Weka e o MOA (o framework mais utilizado em Concept Drift).

# Objetivo deste Notebook

Neste notebook serão feitos alguns testes com uma base de dados bastante utilizada na literatura, a de tipos de floresta. Ela é uma base de dados real e contém mais de 500.000 instâncias, 54 características e sete classes.

Testaremos o método Drift Detection Model (DDM), que monitora o erro das instâncias que chegam. Quando o erro sobe em determinado nível, tem-se o chamado "nível de aviso". Parece que os dados estão começando a mudar. Quando o erro sobe mais, até o chamado "nível de drift", o DDM detectou uma mudança, e o modelo será retreinado a partir do nível de aviso.

Em seguida, veremos a acurácia do modelo sem o DDM, e compararemos a acurácia dos dois. Depois veremos outro método para lidar com Concept Drift, o método do janelamento, em que basicamente o modelo será atualizado depois de determinada quantidade de instâncias entrarem.

Neste notebook não será feita uma análise aprofundada da base de dados. Num futuro próximo farei isso, mas de momento o objetivo é de comparar os métodos descritos.

In [1]:
# Base de dados
import pandas as pd
df = pd.read_csv('datasets/covtype.csv')

In [2]:
df

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type
0,2596,51,3,258,0,510,221,232,148,6279,...,0,0,0,0,0,0,0,0,0,5
1,2590,56,2,212,-6,390,220,235,151,6225,...,0,0,0,0,0,0,0,0,0,5
2,2804,139,9,268,65,3180,234,238,135,6121,...,0,0,0,0,0,0,0,0,0,2
3,2785,155,18,242,118,3090,238,238,122,6211,...,0,0,0,0,0,0,0,0,0,2
4,2595,45,2,153,-1,391,220,234,150,6172,...,0,0,0,0,0,0,0,0,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581007,2396,153,20,85,17,108,240,237,118,837,...,0,0,0,0,0,0,0,0,0,3
581008,2391,152,19,67,12,95,240,237,119,845,...,0,0,0,0,0,0,0,0,0,3
581009,2386,159,17,60,7,90,236,241,130,854,...,0,0,0,0,0,0,0,0,0,3
581010,2384,170,15,60,5,90,230,245,143,864,...,0,0,0,0,0,0,0,0,0,3


# Com detecção de Drift (DDM)

In [3]:
from skmultiflow.drift_detection import DDM

## Tratando a base de dados

In [4]:
import numpy as np

In [5]:
X = df.loc[:, :].drop('Cover_Type', axis=1)

In [6]:
y = df.loc[:, 'Cover_Type']

In [7]:
# Colocando as instâncias para array numpy
X = np.asarray(X)
y = np.asarray(y)

## Árvore de Decisão

In [8]:
# Criando o modelo para as predições

from sklearn.tree import DecisionTreeClassifier

# Criando uma lista para guardar as acurácias
accs = []

# Primeiras instâncias disponíveis
index = 2000

# Adicionando os elementos ao DDM e verificando se ocorreu um nível de aviso ou um drift
for i in range(0, 10):
    print(f"Começando estágio {i+1}...")
    ddm = DDM()
    warn = 0
    right = 0
    dct = DecisionTreeClassifier(max_depth=4)
    dct.fit(X[0:index], y[0:index])
    for j in range(2000, len(X)):
        pred = dct.predict([X[j]])
        if pred == y[j]:
            right += 1

        ddm.add_element(pred == y[j])
        if ddm.detected_warning_zone() and warn == 0:
            print('Warning zone has been detected in data: ' + str(y[j]) + ' - of index: ' + str(j))
            warn = 1
            warn_index = j
        if ddm.detected_change() and (len(np.unique(y[index:j])) > 1):
            print('Change has been detected in data: ' + str(y[j]) + ' - of index: ' + str(j))
            print("Retraining model...\n")
            dct = DecisionTreeClassifier(max_depth=4)
            dct.fit(X[warn_index:j], y[warn_index:j])
            warn = 0
    accs.append(right/len(X))

Começando estágio 1...
Warning zone has been detected in data: 2 - of index: 2046
Change has been detected in data: 7 - of index: 6524
Retraining model...

Warning zone has been detected in data: 7 - of index: 6580
Change has been detected in data: 1 - of index: 59058
Retraining model...

Warning zone has been detected in data: 2 - of index: 59120
Change has been detected in data: 2 - of index: 59140
Retraining model...

Warning zone has been detected in data: 2 - of index: 59248
Change has been detected in data: 2 - of index: 59431
Retraining model...

Warning zone has been detected in data: 2 - of index: 59553
Change has been detected in data: 1 - of index: 59600
Retraining model...

Warning zone has been detected in data: 2 - of index: 59821
Change has been detected in data: 2 - of index: 59834
Retraining model...

Warning zone has been detected in data: 2 - of index: 59958
Change has been detected in data: 2 - of index: 59967
Retraining model...

Warning zone has been detected in d

Warning zone has been detected in data: 1 - of index: 202222
Change has been detected in data: 1 - of index: 202240
Retraining model...

Warning zone has been detected in data: 1 - of index: 202358
Change has been detected in data: 1 - of index: 202531
Retraining model...

Warning zone has been detected in data: 1 - of index: 202606
Change has been detected in data: 2 - of index: 303009
Retraining model...

Warning zone has been detected in data: 2 - of index: 303147
Change has been detected in data: 2 - of index: 303612
Retraining model...

Warning zone has been detected in data: 2 - of index: 304170
Change has been detected in data: 2 - of index: 484237
Retraining model...

Warning zone has been detected in data: 1 - of index: 484326
Change has been detected in data: 1 - of index: 484392
Retraining model...

Warning zone has been detected in data: 2 - of index: 484483
Change has been detected in data: 2 - of index: 484651
Retraining model...

Warning zone has been detected in data: 2

Warning zone has been detected in data: 2 - of index: 528101
Change has been detected in data: 2 - of index: 528220
Retraining model...

Warning zone has been detected in data: 1 - of index: 528515
Change has been detected in data: 1 - of index: 528526
Retraining model...

Warning zone has been detected in data: 1 - of index: 528580
Change has been detected in data: 1 - of index: 528585
Retraining model...

Warning zone has been detected in data: 1 - of index: 528793
Change has been detected in data: 1 - of index: 528808
Retraining model...

Warning zone has been detected in data: 1 - of index: 529090
Change has been detected in data: 1 - of index: 529313
Retraining model...

Change has been detected in data: 1 - of index: 529379
Retraining model...

Warning zone has been detected in data: 1 - of index: 529416
Change has been detected in data: 1 - of index: 529429
Retraining model...

Warning zone has been detected in data: 1 - of index: 529503
Change has been detected in data: 1 - of 

Change has been detected in data: 1 - of index: 539410
Retraining model...

Warning zone has been detected in data: 1 - of index: 539652
Change has been detected in data: 1 - of index: 539671
Retraining model...

Warning zone has been detected in data: 1 - of index: 539906
Change has been detected in data: 1 - of index: 539923
Retraining model...

Warning zone has been detected in data: 1 - of index: 540158
Change has been detected in data: 1 - of index: 540176
Retraining model...

Warning zone has been detected in data: 1 - of index: 540412
Change has been detected in data: 1 - of index: 540429
Retraining model...

Warning zone has been detected in data: 1 - of index: 540658
Change has been detected in data: 1 - of index: 540675
Retraining model...

Warning zone has been detected in data: 1 - of index: 540910
Change has been detected in data: 1 - of index: 540926
Retraining model...

Warning zone has been detected in data: 1 - of index: 541158
Change has been detected in data: 1 - of 

Change has been detected in data: 2 - of index: 93475
Retraining model...

Warning zone has been detected in data: 2 - of index: 93515
Change has been detected in data: 2 - of index: 93566
Retraining model...

Warning zone has been detected in data: 2 - of index: 93612
Change has been detected in data: 2 - of index: 476924
Retraining model...

Warning zone has been detected in data: 2 - of index: 476971
Change has been detected in data: 7 - of index: 477044
Retraining model...

Warning zone has been detected in data: 1 - of index: 477122
Change has been detected in data: 1 - of index: 477416
Retraining model...

Warning zone has been detected in data: 2 - of index: 477481
Change has been detected in data: 2 - of index: 484235
Retraining model...

Warning zone has been detected in data: 7 - of index: 484275
Change has been detected in data: 2 - of index: 484318
Retraining model...

Warning zone has been detected in data: 2 - of index: 484356
Change has been detected in data: 2 - of inde

...

Warning zone has been detected in data: 2 - of index: 397864
Change has been detected in data: 2 - of index: 397893
Retraining model...

Warning zone has been detected in data: 1 - of index: 397928
Change has been detected in data: 1 - of index: 397933
Retraining model...

Warning zone has been detected in data: 1 - of index: 398182
Change has been detected in data: 1 - of index: 398187
Retraining model...

Warning zone has been detected in data: 1 - of index: 398335
Change has been detected in data: 1 - of index: 398387
Retraining model...

Warning zone has been detected in data: 1 - of index: 398551
Change has been detected in data: 1 - of index: 398630
Retraining model...

Warning zone has been detected in data: 2 - of index: 398722
Change has been detected in data: 1 - of index: 398880
Retraining model...

Warning zone has been detected in data: 1 - of index: 399018
Change has been detected in data: 1 - of index: 399088
Retraining model...

Warning zone has been detected in data: 1

Warning zone has been detected in data: 1 - of index: 410587
Change has been detected in data: 1 - of index: 410604
Retraining model...

Warning zone has been detected in data: 1 - of index: 410780
Change has been detected in data: 1 - of index: 410976
Retraining model...

Warning zone has been detected in data: 1 - of index: 411173
Change has been detected in data: 1 - of index: 411180
Retraining model...

Warning zone has been detected in data: 1 - of index: 411292
Change has been detected in data: 1 - of index: 411311
Retraining model...

Warning zone has been detected in data: 1 - of index: 411465
Change has been detected in data: 1 - of index: 412049
Retraining model...

Warning zone has been detected in data: 1 - of index: 412138
Change has been detected in data: 1 - of index: 412176
Retraining model...

Warning zone has been detected in data: 1 - of index: 412279
Change has been detected in data: 1 - of index: 412314
Retraining model...

Warning zone has been detected in data: 1

Warning zone has been detected in data: 1 - of index: 515183
Change has been detected in data: 1 - of index: 515198
Retraining model...

Warning zone has been detected in data: 1 - of index: 515241
Change has been detected in data: 1 - of index: 515248
Retraining model...

Warning zone has been detected in data: 1 - of index: 515498
Change has been detected in data: 1 - of index: 515744
Retraining model...

Warning zone has been detected in data: 1 - of index: 515885
Change has been detected in data: 1 - of index: 515999
Retraining model...

Warning zone has been detected in data: 1 - of index: 516059
Change has been detected in data: 1 - of index: 516075
Retraining model...

Warning zone has been detected in data: 1 - of index: 516115
Change has been detected in data: 1 - of index: 516187
Retraining model...

Warning zone has been detected in data: 2 - of index: 516409
Change has been detected in data: 2 - of index: 516442
Retraining model...

Warning zone has been detected in data: 2

Change has been detected in data: 1 - of index: 526221
Retraining model...

Warning zone has been detected in data: 1 - of index: 526386
Change has been detected in data: 1 - of index: 526444
Retraining model...

Warning zone has been detected in data: 1 - of index: 526485
Change has been detected in data: 1 - of index: 526622
Retraining model...

Warning zone has been detected in data: 1 - of index: 526692
Change has been detected in data: 1 - of index: 526703
Retraining model...

Warning zone has been detected in data: 1 - of index: 526912
Change has been detected in data: 1 - of index: 526948
Retraining model...

Warning zone has been detected in data: 1 - of index: 527388
Change has been detected in data: 1 - of index: 527415
Retraining model...

Warning zone has been detected in data: 1 - of index: 527691
Change has been detected in data: 1 - of index: 527721
Retraining model...

Warning zone has been detected in data: 1 - of index: 527810
Change has been detected in data: 1 - of 

Change has been detected in data: 1 - of index: 537410
Retraining model...

Warning zone has been detected in data: 1 - of index: 537467
Change has been detected in data: 1 - of index: 537470
Retraining model...

Warning zone has been detected in data: 1 - of index: 537552
Change has been detected in data: 1 - of index: 537750
Retraining model...

Warning zone has been detected in data: 1 - of index: 537835
Change has been detected in data: 1 - of index: 538045
Retraining model...

Warning zone has been detected in data: 1 - of index: 538125
Change has been detected in data: 1 - of index: 538322
Retraining model...

Warning zone has been detected in data: 7 - of index: 539503
Change has been detected in data: 2 - of index: 556735
Retraining model...

Warning zone has been detected in data: 2 - of index: 563600
Change has been detected in data: 2 - of index: 565282
Retraining model...

Warning zone has been detected in data: 2 - of index: 565396


In [9]:
accs

[0.4734635429216608,
 0.46595939498667843,
 0.48444610438338626,
 0.5079223837029183,
 0.5056057361982197,
 0.5109102737981316,
 0.5434775873820162,
 0.550353521097671,
 0.5354261185655373,
 0.4935044370856368]

In [10]:
acc_dct = np.mean(accs)

In [11]:
print(f'A acurácia média da Árvore de Decisão depois de 10 passagens pela base de dados foi de: {acc_dct:.2f}')

A acurácia média da Árvore de Decisão depois de 10 passagens pela base de dados foi de: 0.51


# SVM

In [12]:
# Importando o modelo
from sklearn.svm import SVC

ddm = DDM()

# Variáveis de apoio
war = 0
right = 0

# Criando o modelo
svm = SVC()

# Treinamento inicial
svm.fit(X[0:2000], y[0:2000])

# Dados entrando
for j in range(2000, len(X)):
    pred = svm.predict([X[j]])
    if pred == y[j]:
        right += 1

    ddm.add_element(pred == y[j])

    # Checar se o erro chegou ao nível de aviso
    if ddm.detected_warning_zone() and war == 0:
        print('Warning zone has been detected in data: ' + str(y[j]) + ' - of index: ' + str(j))
        war = 1
        war_index = j

    # Checar se o erro chegou ao nível de drift
    if ddm.detected_change() and (len(np.unique(y[war_index:j])) > 2):
        print('Change has been detected in data: ' + str(y[j]) + ' - of index: ' + str(j))
        print("Retraining model...\n")
        svm = SVC()
        svm.fit(X[war_index:j], y[war_index:j])
        war = 0

Warning zone has been detected in data: 2 - of index: 2078
Change has been detected in data: 5 - of index: 6515
Retraining model...

Warning zone has been detected in data: 2 - of index: 6592
Change has been detected in data: 5 - of index: 6648
Retraining model...

Warning zone has been detected in data: 1 - of index: 6695
Change has been detected in data: 1 - of index: 8835
Retraining model...

Warning zone has been detected in data: 5 - of index: 8876
Change has been detected in data: 7 - of index: 9092
Retraining model...

Warning zone has been detected in data: 1 - of index: 9159
Change has been detected in data: 7 - of index: 9215
Retraining model...

Warning zone has been detected in data: 1 - of index: 14335
Change has been detected in data: 7 - of index: 14391
Retraining model...

Warning zone has been detected in data: 1 - of index: 23322
Change has been detected in data: 1 - of index: 63084
Retraining model...

Warning zone has been detected in data: 2 - of index: 63245
Chang

Change has been detected in data: 2 - of index: 323148
Retraining model...

Warning zone has been detected in data: 2 - of index: 323321
Change has been detected in data: 1 - of index: 323464
Retraining model...

Warning zone has been detected in data: 2 - of index: 323631
Change has been detected in data: 2 - of index: 323808
Retraining model...

Warning zone has been detected in data: 1 - of index: 323861
Change has been detected in data: 1 - of index: 324245
Retraining model...

Warning zone has been detected in data: 2 - of index: 324319
Change has been detected in data: 2 - of index: 324499
Retraining model...

Warning zone has been detected in data: 2 - of index: 324538
Change has been detected in data: 1 - of index: 324684
Retraining model...

Warning zone has been detected in data: 2 - of index: 324806
Change has been detected in data: 1 - of index: 324942
Retraining model...

Warning zone has been detected in data: 2 - of index: 325013
Change has been detected in data: 2 - of 

Warning zone has been detected in data: 2 - of index: 347248
Change has been detected in data: 2 - of index: 347594
Retraining model...

Warning zone has been detected in data: 1 - of index: 347689
Change has been detected in data: 2 - of index: 348163
Retraining model...

Warning zone has been detected in data: 1 - of index: 348262
Change has been detected in data: 1 - of index: 348918
Retraining model...

Warning zone has been detected in data: 2 - of index: 348976
Change has been detected in data: 1 - of index: 349146
Retraining model...

Warning zone has been detected in data: 2 - of index: 349238
Change has been detected in data: 2 - of index: 349290
Retraining model...

Warning zone has been detected in data: 2 - of index: 349556
Change has been detected in data: 2 - of index: 349803
Retraining model...

Warning zone has been detected in data: 2 - of index: 349886
Change has been detected in data: 1 - of index: 350056
Retraining model...

Warning zone has been detected in data: 2

Change has been detected in data: 1 - of index: 410111
Retraining model...

Warning zone has been detected in data: 1 - of index: 410256
Change has been detected in data: 1 - of index: 410674
Retraining model...

Warning zone has been detected in data: 1 - of index: 410749
Change has been detected in data: 1 - of index: 410952
Retraining model...

Warning zone has been detected in data: 1 - of index: 411175
Change has been detected in data: 1 - of index: 411309
Retraining model...

Warning zone has been detected in data: 1 - of index: 411466
Change has been detected in data: 1 - of index: 411621
Retraining model...

Warning zone has been detected in data: 1 - of index: 411874
Change has been detected in data: 1 - of index: 412002
Retraining model...

Warning zone has been detected in data: 1 - of index: 412157
Change has been detected in data: 1 - of index: 412323
Retraining model...

Warning zone has been detected in data: 1 - of index: 412566
Change has been detected in data: 1 - of 

Warning zone has been detected in data: 1 - of index: 502234
Change has been detected in data: 1 - of index: 502500
Retraining model...

Change has been detected in data: 1 - of index: 502573
Retraining model...

Warning zone has been detected in data: 1 - of index: 502649
Change has been detected in data: 1 - of index: 503024
Retraining model...

Warning zone has been detected in data: 1 - of index: 503202
Change has been detected in data: 2 - of index: 503312
Retraining model...

Warning zone has been detected in data: 2 - of index: 503357
Change has been detected in data: 1 - of index: 503589
Retraining model...

Change has been detected in data: 1 - of index: 503677
Retraining model...

Warning zone has been detected in data: 1 - of index: 503740
Change has been detected in data: 1 - of index: 504124
Retraining model...

Warning zone has been detected in data: 1 - of index: 504334
Change has been detected in data: 1 - of index: 504700
Retraining model...

Warning zone has been dete

Change has been detected in data: 1 - of index: 524620
Retraining model...

Warning zone has been detected in data: 1 - of index: 524788
Change has been detected in data: 1 - of index: 525085
Retraining model...

Warning zone has been detected in data: 1 - of index: 525270
Change has been detected in data: 1 - of index: 525464
Retraining model...

Warning zone has been detected in data: 1 - of index: 525511
Change has been detected in data: 1 - of index: 526103
Retraining model...

Warning zone has been detected in data: 1 - of index: 526222
Change has been detected in data: 1 - of index: 526500
Retraining model...

Warning zone has been detected in data: 1 - of index: 526714
Change has been detected in data: 1 - of index: 527038
Retraining model...

Warning zone has been detected in data: 1 - of index: 527189
Change has been detected in data: 1 - of index: 527484
Retraining model...

Warning zone has been detected in data: 1 - of index: 527674
Change has been detected in data: 1 - of 

In [13]:
acc_svm = right/len(X)

In [14]:
print(f'A acurácia da SVM com DDM foi de: {acc_svm}')

A acurácia da SVM com DDM foi de: 0.5057864553572043


# Sem detecção de Drift

Aqui serão feitos dois testes: O primeiro, treinar o modelo em uma certa quantidade de dados, e depois testá-lo em dados que estão à frente na linha do tempo. O segundo será atualizar o modelo sem nenhum método de detecção de drift. Será definida um janela e o modelo será atualizado de acordo com o tamanho dessa janela.

In [15]:
dct = DecisionTreeClassifier(max_depth=4)
dct.fit(X[0:90000], y[0:90000])

DecisionTreeClassifier(max_depth=4)

In [16]:
preds = dct.predict(X[90000:])

In [17]:
from sklearn.metrics import accuracy_score

accuracy_score(preds, y[90000:])

0.4796318623577428

Vemos aqui que, depois de treinar nas 90000 primeiras instâncias, o modelo não consegue predizer bem as seguintes, obtendo um desempenho inferior aos modelos anteriores com detecção de drift.

Agora vamos testar o método do janelamento

## Janelamento

In [18]:
# Definindo o tamanho da janela
window_size = 2000

# Criando uma árvore de decisão
dct = DecisionTreeClassifier(max_depth=4)

# Treinando a árvore de decisão na primeira janela
dct.fit(X[:window_size], y[:window_size])

DecisionTreeClassifier(max_depth=4)

In [19]:
# Barra de progresso
import progressbar

bar = progressbar.ProgressBar(max_value=len(X))

In [20]:
# Avaliando e atualizando a cada janela

window_beg = window_size
counter = 0
right = 0

for i in range(window_size, len(X)):
    if counter == window_size:
        window_end = i
        dct = DecisionTreeClassifier(max_depth=4)
        dct.fit(X[window_beg:window_end], y[window_beg:window_end])
        counter = 0
    pred = dct.predict([X[i]])
    if pred == y[i]:
        right += 1
        
    bar.update(i+1)
    counter += 1

100% (581012 of 581012) |################| Elapsed Time: 0:04:10 ETA:  00:00:00

In [21]:
right / len(X)

0.6801976551258838

E veja só, o melhor resultado até agora. Vamos tentar de novo, dessa vez com uma janela maior:

In [22]:
# Definindo o tamanho da janela
window_size = 20000

# Criando uma árvore de decisão
dct = DecisionTreeClassifier(max_depth=4)

# Treinando a árvore de decisão na primeira janela
dct.fit(X[:window_size], y[:window_size])

bar = progressbar.ProgressBar(max_value=len(X))

# Avaliando e atualizando a cada janela

window_beg = window_size
counter = 0
right = 0

for i in range(window_size, len(X)):
    if counter == window_size:
        window_end = i
        dct = DecisionTreeClassifier(max_depth=4)
        dct.fit(X[window_beg:window_end], y[window_beg:window_end])
        counter = 0
    pred = dct.predict([X[i]])
    if pred == y[i]:
        right += 1
        
    bar.update(i+1)
    counter += 1

100% (581012 of 581012) |################| Elapsed Time: 0:00:49 ETA:  00:00:00

In [24]:
right / len(X)

0.647026223210536

E assim vemos que, não necessariamente, uma janela maior dará o melhor resultado. Se a janela foi muito grande, é possível que o modelo demore a se adaptar a um conceito. Já se for muito pequena, o modelo não terá sido treinado em instâncias suficientes, o que pode comprometer no desempenho. A partir daqui, começarei a fazer testes com outros métodos disponíveis na literatura e voltarei para atualizar este notebook.

Por agora, o que podemos concluir é: se utilizamos algums método para lidar com Concept Drift, na base de dados apresentada neste notebook, ele terá um desempenho melhor que se apenas treinarmos o modelo nas instâncias disponíveis e testá-lo nas subsequentes (lembrando, no cenário de Data Streams as instâncias não estão todas disponíveis a princípio). O método do janelamento foi o que obteve o melhor desempenho.

# Obrigado! Que a força esteja com você.